In [1]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW
from xgboost import XGBClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle as pkl
import scipy
import os
import gurobipy as gb

from torch.nn import Linear, ReLU, Dropout
from torch.nn.functional import relu
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [2]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = (
        False  # Force cuDNN to use a consistent convolution algorithm
    )
    torch.backends.cudnn.deterministic = (
        True  # Force cuDNN to use deterministic algorithms if available
    )
    torch.use_deterministic_algorithms(
        True
    )  # Force torch to use deterministic algorithms if available


In [3]:
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seeds(42)

cuda:0


In [4]:
config = {
        'train_val_split': [0.80, 0.20], # These must sum to 1.0
        'batch_size' : 32, # Num samples to average over for gradient updates
        'EPOCHS' : 1000, # Num times to iterate over the entire dataset
        'LEARNING_RATE' : 1e-3, # Learning rate for the optimizer
        'BETA1' : 0.9, # Beta1 parameter for the Adam optimizer
        'BETA2' : 0.999, # Beta2 parameter for the Adam optimizer
        'WEIGHT_DECAY' : 1e-4, # Weight decay parameter for the Adam optimizer
    }

In [5]:
try:
    corlat_dataset = pkl.load(open("Data/corlat/corlat_preprocessed_v1.pickle", "rb"))
except:
    # move dir to /ibm/gpfs/home/yjin0055/Project/DayAheadForecast
    os.chdir("/ibm/gpfs/home/yjin0055/Project/DayAheadForecast")
    corlat_dataset = pkl.load(open("Data/corlat/corlat_preprocessed_v1.pickle", "rb"))

In [6]:
corlat_dataset[0]["input"]["var_node_features"].head()

,var_obj_coef,num_nonzero_coef,lp_relax_val,is_lp_relax_val_frac,lp_sol_val_eq_lb,lp_sol_val_eq_ub,has_lb,has_ub,mean_degree,std_degree,min_degree,max_degree,mean_coef,std_coef,min_coef,max_coef,var_type_B,var_type_C
0,5.0,6.0,1.000000,True,True,True,True,True,19.000000,36.706039,1.0,101.0,-50.166667,49.837792,-100.0,1.0,1,0
1,4.0,6.0,0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0,1,0
2,6.0,6.0,0.442327,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-50.166667,49.837792,-100.0,1.0,1,0
3,5.0,6.0,-0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0,1,0
4,3.0,6.0,-0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0,1,0


In [7]:
# Obtain the maximum size of N_constraints and N_variables across the dataset.

max_N_constraints = max(
    len(x["input"]["constraint_node_features"]) for x in corlat_dataset
)

max_N_variables = max(
    len(x["input"]["var_node_features"]) for x in corlat_dataset
)

min_N_constraints = min(
    len(x["input"]["constraint_node_features"]) for x in corlat_dataset
)

min_N_variables = min(
    len(x["input"]["var_node_features"]) for x in corlat_dataset
)

In [8]:
print("Maximum number of variables: ", max_N_variables)
print("Maximum number of constraints: ", max_N_constraints)
print("Minimum number of variables: ", min_N_variables)
print("Minimum number of constraints: ", min_N_constraints)

Maximum number of variables:  466
Maximum number of constraints:  551
Minimum number of variables:  466
Minimum number of constraints:  470


In [9]:
print("Number of variable node features: ", len(corlat_dataset[0]["input"]["var_node_features"].columns))
print("Number of constraint node features: ", len(corlat_dataset[0]["input"]["constraint_node_features"].columns))

Number of variable node features:  18
Number of constraint node features:  10


In [10]:
# for each variable node features, pad with 0.0s to make it the same length as the maximum number of variables

var_node_features = np.stack(
    [
        np.pad(
            x["input"]["var_node_features"].values,
            ((0, max_N_variables - len(x["input"]["var_node_features"])), (0, 0)),
            "constant",
            constant_values=0.0,
        )
        for x in corlat_dataset
    ]
)
            

In [11]:
var_node_features.shape

(2000, 466, 18)

In [12]:
constraint_node_features = np.stack(
    [
        np.pad(
            x["input"]["constraint_node_features"].values,
            ((0, max_N_constraints - len(x["input"]["constraint_node_features"])), (0, 0)),
            "constant",
            constant_values=0.0,
        )
        for x in corlat_dataset
    ]   
)

In [13]:
constraint_node_features.shape

(2000, 551, 10)

In [14]:
# for var_node_features and constraint_node_features, reshape to (N_samples, -1) to feed into the neural network
var_input = var_node_features.reshape(var_node_features.shape[0], -1)
constraint_input = constraint_node_features.reshape(constraint_node_features.shape[0], -1)

In [15]:
print("Shape of variable features input: ", var_input.shape)
print("Shape of constraint features input: ", constraint_input.shape)

Shape of variable features input:  (2000, 8388)
Shape of constraint features input:  (2000, 5510)


In [16]:
# get A matrix input by stacking the csr_matrix of each sample getting shape of N_samples x (A.shape[0] x A.shape[1])
A_input = np.vstack([x["input"]["A"] for x in corlat_dataset])

In [17]:
A_input.shape

(2000, 1)

In [18]:
A_feature_list = []
for i in range(len(corlat_dataset)):
    n_cons = corlat_dataset[i]["input"]["A"].shape[0]

    # for row in range(n_vars):
    #     for col in range(n_cons):
    #         if input_dict_list[i]["A"][row, col] != 0:
    #             adj_matrix[row, n_vars + col] = input_dict_list[i]["A"][row, col]
    #             adj_matrix[n_vars + col, row] = input_dict_list[i]["A"][row, col]

    I, J, V = scipy.sparse.find(corlat_dataset[i]["input"]["A"])
    # adj_matrix[I, n_vars + J] = V
    # adj_matrix[n_vars + J, I] = V

    # # convert to COO format
    edge_index = torch.stack([torch.tensor(I), torch.tensor(n_cons + J)], dim=0)

    # expand V to 2D
    edge_attr = torch.tensor(V).unsqueeze(1)

    tmp_dict = {"edge_index": edge_index, "edge_attr": edge_attr}
    A_feature_list.append(tmp_dict)

In [19]:
# for each sample, pad the edge_index and edge_attr to make it the same length as the maximum length of edge_index and edge_attr
max_edge_index_len = max([len(x["edge_index"][0]) for x in A_feature_list])
max_edge_attr_len = max([len(x["edge_attr"]) for x in A_feature_list])

for i in range(len(A_feature_list)):
    edge_index = A_feature_list[i]["edge_index"]
    edge_attr = A_feature_list[i]["edge_attr"]

    # pad edge_index
    edge_index = torch.cat(
        [
            edge_index,
            torch.zeros(
                2, max_edge_index_len - len(edge_index[0]), dtype=torch.long
            ),
        ],
        dim=1,
    )

    # pad edge_attr
    edge_attr = torch.cat(
        [
            edge_attr,
            torch.zeros(
                max_edge_attr_len - len(edge_attr), 1, dtype=torch.float32
            ),
        ],
        dim=0,
    )

    A_feature_list[i]["edge_index"] = edge_index
    A_feature_list[i]["edge_attr"] = edge_attr

In [20]:
# check if the padding is correct by checking the shape of edge_index and edge_attr
for i in range(len(A_feature_list)):
    assert A_feature_list[i]["edge_index"].shape == (2, max_edge_index_len)
    assert A_feature_list[i]["edge_attr"].shape == (max_edge_attr_len, 1)

In [21]:
print("Shape of A matrix input: ", A_input.shape)

Shape of A matrix input:  (2000, 1)


In [22]:
corlat_dataset[0].keys()

dict_keys(['solution', 'indices', 'input'])

In [23]:
# for each solution convert the dictionary to a list of values
solutions = [
    list(corlat_dataset[i]["solution"].values())
    for i in range(len(corlat_dataset))
]

In [24]:
# convert solutions_list to numpy array
solutions = np.array(solutions)

In [25]:
# combine the variable features and constraint features into a single input
X = np.hstack([var_input, constraint_input])

In [26]:
n_features = X.shape[1]
out_channels = solutions.shape[1]

In [27]:
out_channels

100

In [28]:
X.shape

(2000, 13898)

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(n_features, n_features//4)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_features//4, n_features//2)
        self.fc3 = nn.Linear(n_features//2, n_features//2)
        self.fc4 = nn.Linear(n_features//2, out_channels)
        self.sigmoid = nn.Sigmoid()
        
        # add regularization
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        
        return x

In [30]:
# check type for one sample of solutions
solutions[0].dtype

dtype('float64')

In [31]:
# convert X and solutions to float32
X = X.astype(np.float32)
solutions = solutions.astype(np.float32)

In [32]:
# check all samples have same dimension
for i in range(len(solutions)):
    assert solutions[i].shape == (100,)

for i in range(len(X)):
    assert X[i].shape == (13898,)

In [33]:
# train test split to get indices for train and test
train_idx, test_idx = train_test_split(
    np.arange(len(solutions)), test_size=0.2, random_state=42
)

X_train = X[train_idx]
X_test = X[test_idx]
y_train = solutions[train_idx]
y_test = solutions[test_idx]

In [34]:
# X_train, X_test, y_train, y_test = train_test_split(X, solutions, test_size=0.2, random_state=42, shuffle=True)
# save the train and test data in the directory Data/corlat/
np.save("Data/corlat/X_train_v1.npy", X_train)
np.save("Data/corlat/X_test_v1.npy", X_test)
np.save("Data/corlat/y_train_v1.npy", y_train)
np.save("Data/corlat/y_test_v1.npy", y_test)
np.save("Data/corlat/train_idx_v1.npy", train_idx)
np.save("Data/corlat/test_idx_v1.npy", test_idx)

In [35]:
net = NeuralNetwork()
# net = torch.compile(net)

batch_size = 32

criterion = nn.BCELoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001)

# create the dataloader for X and solutions
train_loader = DataLoader(
    TensorDataset(torch.tensor(X_train), torch.tensor(y_train)),
    batch_size=batch_size,
    shuffle=True,
)

valid_loader = DataLoader(
    TensorDataset(torch.tensor(X_test), torch.tensor(y_test)),
    batch_size=batch_size,
    shuffle=True,
)

params = list(net.parameters())

# optimizer = AdamW(params, lr=config['LEARNING_RATE'], weight_decay=1e-4)
optimizer = optim.SGD(net.parameters(), lr=0.001)
# optimizer = dadaptation.DAdaptAdam(params, lr=1, log_every=5, betas=(BETA1, BETA2), weight_decay=1e-4, decouple=True)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
total_steps = len(train_loader)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config['LEARNING_RATE'], steps_per_epoch=total_steps, epochs=config['EPOCHS'])

In [36]:
net.to(device)

NeuralNetwork(
  (fc1): Linear(in_features=13898, out_features=3474, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=3474, out_features=6949, bias=True)
  (fc3): Linear(in_features=6949, out_features=6949, bias=True)
  (fc4): Linear(in_features=6949, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [37]:
for epoch in range(config["EPOCHS"]):
    running_loss = 0.0
    curr_lr = optimizer.param_groups[0]['lr']
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    print('Epoch %d loss: %.3f lr: %.6f' % (epoch + 1, running_loss / len(train_loader), curr_lr))

Epoch 1 loss: 0.804 lr: 0.000040
Epoch 2 loss: 0.702 lr: 0.000040
Epoch 3 loss: 0.668 lr: 0.000040
Epoch 4 loss: 0.648 lr: 0.000040
Epoch 5 loss: 0.633 lr: 0.000040
Epoch 6 loss: 0.623 lr: 0.000041
Epoch 7 loss: 0.614 lr: 0.000041
Epoch 8 loss: 0.605 lr: 0.000041
Epoch 9 loss: 0.598 lr: 0.000042
Epoch 10 loss: 0.591 lr: 0.000042
Epoch 11 loss: 0.586 lr: 0.000043
Epoch 12 loss: 0.580 lr: 0.000043
Epoch 13 loss: 0.575 lr: 0.000044
Epoch 14 loss: 0.570 lr: 0.000044
Epoch 15 loss: 0.565 lr: 0.000045
Epoch 16 loss: 0.562 lr: 0.000046
Epoch 17 loss: 0.557 lr: 0.000047
Epoch 18 loss: 0.554 lr: 0.000048
Epoch 19 loss: 0.550 lr: 0.000049


KeyboardInterrupt: 

In [125]:
# if X_test is numpy array convert to tensor
if isinstance(X_test, np.ndarray):
    X_test = torch.tensor(X_test)
    
# if net device is cpu convert X_test to cpu, else convert to cuda
if next(net.parameters()).device == torch.device("cpu"):
    X_test = X_test.cpu()
else:
    X_test = X_test.cuda()

In [126]:
# validation of the model using f1 score, precision and recall
y_pred = net(torch.tensor(X_test))
y_pred = y_pred.cpu().detach().numpy()
y_pred = np.where(y_pred > 0.5, 1, 0)

y_test = y_test.astype(np.int)

print("F1 score: ", f1_score(y_test, y_pred, average="micro"))
print("Precision: ", precision_score(y_test, y_pred, average="micro"))
print("Recall: ", recall_score(y_test, y_pred, average="micro"))


F1 score:  0.7200379266750949
Precision:  0.7599066044029353
Recall:  0.6841441441441442


/tmp/ipykernel_767776/837471902.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = net(torch.tensor(X_test))
/tmp/ipykernel_767776/837471902.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = y_test.astype(np.int)


In [132]:
def feasibility_test(batch_size, y_pred, test_models, indices):
    
    n_violated_constraints = []

    # convert predictions of N_samples, N_variables to binary
    y_pred_binary = np.where(y_pred > 0.5, 1, 0)
    
    # Compute the weights for each training instance
    for i in range(len(test_models)):
        
        model = test_models[i]
        
        modelVars = model.getVars()
        
        instanceBinaryIndices = indices

        # need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
        for j in range(len(instanceBinaryIndices)):
            modelVars[instanceBinaryIndices[j]].setAttr("VType", "C")

            # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
            modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_binary[i, j])
            modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_binary[i, j])
        
        
        # Compute the IIS to find the list of violated constraints and variables
        try:
            model.computeIIS()
        except gb.GurobiError:
            print("Model is feasible")
            n_violated_constraints.append(0)
            continue
            
        
        # get number of violated constraints
        IISConstr = model.getAttr("IISConstr", model.getConstrs())

        # count number of non zero elements in IISConstr        
        n_violated_constraints.append(np.count_nonzero(IISConstr))
        
    return n_violated_constraints

In [133]:
test_models = []
gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)
model_files = os.listdir("instances/mip/data/COR-LAT")
for i in range(len(test_idx)):
    model = gb.read("instances/mip/data/COR-LAT/" + model_files[test_idx[i]], env=gurobi_env)
    test_models.append(model)
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-01


In [134]:
n_violated_constraints = []
binary_indices = corlat_dataset[0]["indices"]["indices"]
for i, data in enumerate(valid_loader):
    inputs, labels = data
    
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    outputs = net(inputs)
    
    # get slices of test_models according to batch size
    len_test_models = len(test_models)
    test_models_batch = test_models[i*batch_size: min((i+1)*batch_size, len_test_models)]
    
    n_violated_constraints_batch = feasibility_test(batch_size, outputs.detach().cpu().numpy(), test_models_batch, binary_indices)
    
    n_violated_constraints.append(n_violated_constraints_batch)
    #

Model is feasible
Model is feasible
Model is feasible
Model is feasible


In [135]:
# count average number of violated constraints by flattening the list
n_violated_constraints = [item for sublist in n_violated_constraints for item in sublist]
print("Average number of violated constraints: ", np.mean(n_violated_constraints))


Average number of violated constraints:  3.085


In [136]:
print("Length of n_violated_constraints: ", len(n_violated_constraints))
print(n_violated_constraints)

Length of n_violated_constraints:  400
[1, 1, 36, 17, 1, 1, 1, 1, 1, 1, 2, 1, 8, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 20, 1, 1, 1, 2, 1, 1, 1, 1, 1, 17, 1, 1, 0, 1, 28, 1, 1, 2, 1, 1, 1, 1, 1, 12, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 50, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 1, 19, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 13, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 22, 1, 1, 2, 1, 1, 1, 2, 1, 30, 1, 2, 26, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 42, 1, 8, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 56, 1, 1, 2, 1, 1, 1, 51, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 24, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 5, 30, 27, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 12, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 73, 1, 1, 1, 1, 1, 1, 1

In [137]:
def calculate_diving_opt_time(models, binary_indices, y_pred):
    
    opt_time = []
    
    for i in range(len(models)):
        model = models[i]
        
        modelVars = model.getVars()
        
        instanceBinaryIndices = binary_indices
        
        y_pred_binary = np.where(y_pred > 0.5, 1, 0)
        
        # need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
        for j in range(len(instanceBinaryIndices)):
            modelVars[instanceBinaryIndices[j]].setAttr("VType", "C")

            # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
            modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_binary[i, j])
            modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_binary[i, j])
        
        
        # Compute the IIS to find the list of violated constraints and variables
        try:
            model.computeIIS()
            infeasible_flag = True
        except gb.GurobiError:
            print("Model is feasible")
            infeasible_flag = False
            continue
        
        if infeasible_flag:
            for j in range(len(instanceBinaryIndices)):
                if modelVars[instanceBinaryIndices[j]].IISLB == 0 and modelVars[instanceBinaryIndices[j]].IISUB == 0:
                    modelVars[instanceBinaryIndices[j]].setAttr("VType", "B")
                    # for each index in binary_indices, set the value of the corresponding variable to the value predicted by model
                    modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_binary[i, j])
                    modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_binary[i, j])                     
                    
                    # else if the variable is in the IIS, 
                    # get the relaxed variable and 
                    # set the bounds to 0 and 1 for the relaxed binary variables
                else:
                    modelVars[instanceBinaryIndices[j]].setAttr("VType", "B")
                    modelVars[instanceBinaryIndices[j]].setAttr("LB", 0)
                    modelVars[instanceBinaryIndices[j]].setAttr("UB", 1)
        
        else:
            for j in range(len(instanceBinaryIndices)):
                modelVars[instanceBinaryIndices[j]].setAttr("VType", "B")
                modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_binary[i, j])
                modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_binary[i, j])
        
        model.Params.Threads = 1
        model.optimize()
        print("Optimization time for model ", i, ": ", model.Runtime)
        opt_time.append(model.Runtime)
        
    return opt_time



In [138]:
test_models = []
gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)
model_files = os.listdir("instances/mip/data/COR-LAT")
for i in range(len(test_idx)):
    model = gb.read("instances/mip/data/COR-LAT/" + model_files[test_idx[i]], env=gurobi_env)
    test_models.append(model)
    
# loop through all test models and calculate average optimization time
opt_time = []
for i, data in enumerate(valid_loader):
    inputs, labels = data
    
    inputs = inputs.to(device)
    # labels = labels.to(device)
    
    outputs = net(inputs)
    
    # get slices of test_models according to batch size
    len_test_models = len(test_models)

    test_models_batch = test_models[i*batch_size: min((i+1)*batch_size, len_test_models)]
    
    opt_time_batch = calculate_diving_opt_time(test_models_batch, binary_indices, outputs.detach().cpu().numpy())
    
    opt_time.append(opt_time_batch)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-01
Optimization time for model  0 :  0.0003139972686767578
Optimization time for model  1 :  0.0001800060272216797
Optimization time for model  2 :  0.0001919269561767578
Optimization time for model  3 :  0.0018579959869384766
Optimization time for model  4 :  0.00015401840209960938
Optimization time for model  5 :  0.49094700813293457
Optimization time for model  6 :  0.00018715858459472656
Optimization time for model  7 :  0.0001430511474609375
Optimization time for model  8 :  0.00014495849609375
Optimization time for model  9 :  0.00017690658569335938
Optimization time for model  10 :  0.00014209747314453125
Optimization time for model  11 :  0.00011897087097167969
Optimization time for model  12 :  0.00017213821411132812
Optimization time for model  13 :  0.0001380443572998047
Optimization time for model  14 :  0.144395112991333
Optimization time for model  15 :  0.3048369884490967
Optimization

In [139]:
# flatten opt_time
opt_time_flat = [item for sublist in opt_time for item in sublist]

print("Average optimization time: ", np.mean(opt_time_flat))

Average optimization time:  0.29866028432581826


# XGBoost model

In [44]:
# save neural network model
print("Saving model...")
# statedict
torch.save(net.state_dict(), "Models/Tabular/neural_network_model_corlat.pt")

Saving model...


In [ ]:
# XGBoost model
y_test = y_test.astype(np.int)
y_train = y_train.astype(np.int)
clf = XGBClassifier(tree_method='hist')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/tmp/ipykernel_91040/2751798150.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = y_test.astype(np.int)
/tmp/ipykernel_91040/2751798150.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for

In [40]:
print("F1 score: ", f1_score(y_test, y_pred, average="micro"))
print("Precision: ", precision_score(y_test, y_pred, average="micro"))
print("Recall: ", recall_score(y_test, y_pred, average="micro"))

F1 score:  0.9237003241685676
Precision:  0.9232569302772111
Recall:  0.9241441441441441


In [43]:
# save xgboost model
print("Saving model...")
pkl.dump(clf, open("Models/Tabular/xgboost_model_corlat.pkl", "wb"))

Saving model...


In [47]:
# try to load the model net = NeuralNetwork()
net.load_state_dict(torch.load("Models/Tabular/neural_network_model_corlat.pt"))

<All keys matched successfully>

In [76]:
# load xgboost model
clf = pkl.load(open("Models/Tabular/xgboost_model_corlat.pkl", "rb"))

In [78]:

y_test = y_test.astype(np.int)
y_train = y_train.astype(np.int)
y_pred = clf.predict(X_test.cpu().detach().numpy())

/tmp/ipykernel_767776/551078606.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = y_test.astype(np.int)
/tmp/ipykernel_767776/551078606.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for

In [79]:
print("F1 score: ", f1_score(y_test, y_pred, average="micro"))
print("Precision: ", precision_score(y_test, y_pred, average="micro"))
print("Recall: ", recall_score(y_test, y_pred, average="micro"))

F1 score:  0.9237003241685676
Precision:  0.9232569302772111
Recall:  0.9241441441441441


In [81]:
binary_indices = corlat_dataset[0]["indices"]["indices"]
    
n_violated_constraints = feasibility_test(len(y_pred), y_pred, test_models, binary_indices)    #

Model is feasible
Model is feasible
Model is feasible
Model is feasible
Model is feasible
Model is feasible
Model is feasible
Model is feasible
Model is feasible


In [83]:
print("Average number of violated constraints: ", np.mean(n_violated_constraints))

Average number of violated constraints:  3.78
